# Keras ImageDataGenerator

The methods of image normalization using keras ImageDataGenerator are analyzed.

## Download MNIST dataset

In [ ]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils    import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import numpy as np
np.set_printoptions(precision=4, suppress=True)

In [ ]:
# load dataset
(X_trn, Y_trn), (X_tst, Y_tst) = mnist.load_data()

# summarize dataset shape
print('Train: images:', X_trn.shape, 'classes:', Y_trn.shape)
print('Test : images:', X_tst.shape, 'classes:', Y_tst.shape)

# summarize pixel values
print(f"Train:  min:{X_trn.min()}, max:{X_trn.max()} mean:{X_trn.mean():.1f}, std:{X_trn.std():.1f}")
print(f"Test:   min:{X_tst.min()}, max:{X_tst.max()} mean:{X_tst.mean():.1f}, std:{X_tst.std():.1f}")

## Used ImageDataGenerator 

In [ ]:
IMAGE_SHAPE = (X_tst.shape[1], X_tst.shape[2], 1)

def get_data(X,Y, batch_size):
    X = X.reshape( (-1, ) +  IMAGE_SHAPE )
    Y = to_categorical(Y)                               # one hot encode target values    
    print("X:",X.shape, " Y:",Y.shape)

    gen =   ImageDataGenerator(
                rescale = 1.0/255,                     # 0) X *= rescale
        
                #samplewise_center  = True,             # 1) Set each sample mean to 0 (False)
                #samplewise_std_normalization  = True,  # 2) Divide each input by its std (False)                                                 
        
                featurewise_center = True,             # 3) Set input mean to 0 over the dataset
                featurewise_std_normalization = True,  # 4) Divide inputs by std of the dataset, feature-wise (False)
            )                             
                                          
    gen.fit(X)                                         # only for featurewise !!!
    
    gen_iter = gen.flow(X, Y, batch_size=batch_size)   # prepare an iterators to scale images    
    
    return gen_iter, gen

gen_iter, gen = get_data(X_tst, Y_tst, 100)


print(f"gen     mean: {gen.mean}, std: {gen.std}")
print(f"dataset mean: {(X_tst/255).mean():.4f}, std: {(X_tst/255).std():.4f}\n")

batch_X, batch_Y = gen_iter.next()

print('batches:', len(gen_iter), end="   ")                           # 10000/batch_size 
print(f'batch shape: {batch_X.shape}, min: {batch_X.min():.3f}, max:{batch_X.max():.3f}')
print(f'batch mean: {batch_X.mean():.3f}')
print(f'batch std : {batch_X.std():.3f}\n')
print('mean:', batch_X[0:8].mean( (1,2,3) ))
print('std :', batch_X[0:8].std ( (1,2,3) ))
if gen.mean is not None:
    print('mean(x_f):', -gen.mean/gen.std,  'std(x_f):', 1./gen.std, "(see below explanation)")

## Conclusions

- if using only **samplewise**, each image will have zero mean and unit variance
- if using only **featurewise**, each image will have non zero mean and non unit variance, but on average the batch will be approximately normalized
- if you use **samplewise + featurewise** at the same time, each image will be highly biased and have a large variance (if the entire dataset has not been normalized). Thus, their simultaneous use is impractical.


## Explanation

https://github.com/keras-team/keras/blob/v2.9.0/keras/preprocessing/image.py 

Keras first calculates statistics for the entire dataset (fit function).  `x.shape = (N,C,W,H)`
```
fit(x):                                     # Fits the data generator to some sample data.
    self.mean = np.mean(x, axis=(0, 2, 3))  # mean by each channel throughout the dataset       
    self.std  = np.std (x, axis=(0, 2, 3))  # if there is only one channel, then it is mean(), std()
```
Next (samplewise) it nnormalizes each image by its mean and variance.<br>
_After that_ (featurewise) normalizes the result by global statistics (and gets the offset values in each image !!!?)
```
standardize(x):                            # Applies the normalization configuration in-place to a batch of inputs.
    if self.rescale:                       x *= self.rescale
    if self.samplewise_center:             x -= np.mean(x, keepdims=True)  # to one image
    if self.samplewise_std_normalization:  x /= (np.std(x, keepdims=True) + 1e-6)
    if self.featurewise_center:            x -= self.mean
    if self.featurewise_std_normalization: x /= (self.std + 1e-6)
          
batch_x[i] = self.image_data_generator.standardize(x)
```

Throughout the dataset:
$$
m_f = \bar{x}_\mathrm{dataset},~~~~\sigma_f=\mathrm{std}(x_\mathrm{dataset})
$$
For each image $x$:
$$
x ~~~\mapsto~~~x_s = \frac{x-\bar{x}}{\sigma}\sim\mathcal{N}(0,1)~~~\mapsto~~~   x_f  = \frac{x_s - m_f}{\sigma_f} 
~~~~\Rightarrow~~~~~
\langle x_f \rangle = -\frac{m_f}{\sigma_f},~~~~~~~~~~\mathrm{std}(x_f)= \frac{1}{\sigma_f} 
$$

In fact, the convolutional layer copes well with the bias (MNIST):
```
Test Accuracy: 98.720,  98.990   only rescale
Test Accuracy: 98.990,  98.950   samplewise + featurewise
Test Accuracy: 99.030,  98.900   samplewise
```

## Train MNIST

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, BatchNormalization, Input

iter_trn, gen_trn = get_data(X_trn, Y_trn, 64)
iter_tst, gen_tst = get_data(X_tst, Y_tst, 64)

# define model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=IMAGE_SHAPE))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(10, activation='softmax'))

model.summary()

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit_generator(iter_trn, steps_per_epoch=len(iter_trn), epochs=5)         # fit model with generator
_, acc = model.evaluate_generator(iter_tst, steps=len(iter_tst), verbose=0)      # evaluate model

print('Test Accuracy: %.3f' % (acc * 100))

# Data augmentation

In [ ]:
import matplotlib.pyplot as plt

IMAGE_SHAPE = (X_tst.shape[1], X_tst.shape[2], 1)

def plot_samples(X, cols = 10, rows = 2):            
    plt.figure(figsize=(2*cols, 2*rows))     
    num = min(cols*rows, len(X))
    for i in range(num):
        im = X[i]
        mi, ma = im.min(), im.max()        
        plt.subplot(rows, cols, i + 1)        
        plt.imshow(im, cmap='gray', vmin=mi, vmax=ma)
        plt.axis('off')        
    plt.show()

def get_data(X, Y, batch_size):
    X = X.reshape( (-1, ) +  IMAGE_SHAPE )        

    gen =   ImageDataGenerator(
                rescale = 1.0/255,                     # X *= rescale
        
                samplewise_center  = True,             # Set each sample mean to 0 (False)
                samplewise_std_normalization  = True,  # Divide each input by its std (False)                                                 

                vertical_flip=True, 
                horizontal_flip=True,                                              
                rotation_range=180.0, 
                brightness_range=(0.8,1.2)        
            )                                                                           
    
    gen_iter = gen.flow(X, Y, batch_size=batch_size)   # prepare an iterators to scale images    
    
    return gen_iter, gen

plot_samples(X_tst)

gen_iter, gen = get_data(X_tst,Y_tst, 100)

batch_X, batch_Y = gen_iter.next()

print("augmentation")
plot_samples(batch_X)

In [22]:
import numpy as np
from tensorflow.keras import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Input

x = Input(shape=(16,16,1))
y = Conv2D(1, (1, 1), activation='relu')(x)
          
model = Model(inputs=x, outputs=y)
model.summary()

out = model( np.zeros( (16,16,1) ) )
print(out.shape)

Model: "functional_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_21 (InputLayer)        [(None, 16, 16, 1)]       0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 16, 16, 1)         2         
Total params: 2
Trainable params: 2
Non-trainable params: 0
_________________________________________________________________
(16, 16, 1, 1)
